In [10]:
# imports
import os
import cv2
#import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import sklearn as skl
import sklearn.preprocessing as preproc
import numpy as np
import keras as k
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, ZeroPadding2D, Flatten, Dense
import matplotlib.pyplot as plt
from sklearn.ensemble import ExtraTreesClassifier
from time import time
import pandas as pd

In [25]:
# carica le immagini e le label
def loadPictures(path, csv):
    X=[];
    y=[];
    
    dataset = pd.read_csv(csv, encoding = "ISO-8859-1") #N.B: questo noisy è corretto a mano
    
    for _,_,files in os.walk(path+"/"):
        for file in files:
            #print(file)
            if file.find('.jpg')<0:
                continue     
            img = cv2.imread(path+"/"+file)
            X.append(img)
            y.append(dataset[dataset['image_id'] == file]['dx'])
    return X,y

In [26]:
loadPictures('..\..\skin-cancer-mnist-ham10000\HAM10000_images_part_1','HAM10000_metadata.csv')
#path = '..\..\skin-cancer-mnist-ham10000\HAM10000_images_part_1'
#print(os.walk(path))
#os.walk(path)

([array([[[131, 131, 221],
          [126, 128, 222],
          [127, 129, 223],
          ...,
          [138, 134, 229],
          [138, 136, 226],
          [139, 135, 230]],
  
         [[129, 129, 223],
          [126, 130, 225],
          [133, 132, 228],
          ...,
          [137, 135, 225],
          [136, 136, 226],
          [135, 134, 226]],
  
         [[126, 128, 222],
          [131, 129, 229],
          [134, 130, 231],
          ...,
          [132, 136, 224],
          [133, 136, 227],
          [134, 137, 228]],
  
         ...,
  
         [[125, 118, 209],
          [126, 119, 210],
          [126, 121, 212],
          ...,
          [128, 140, 210],
          [130, 141, 209],
          [128, 139, 207]],
  
         [[118, 116, 205],
          [121, 119, 208],
          [124, 120, 209],
          ...,
          [129, 142, 210],
          [131, 144, 212],
          [130, 143, 211]],
  
         [[115, 116, 207],
          [116, 120, 209],
          [118, 120, 208

In [28]:
y[0]

NameError: name 'y' is not defined